# 第三部分：PyTorch与深度学习基础
本项目是**中南大学智能雷达实验室**为即将进组的同学准备的培训项目，涵盖从Python基础到深度学习在探地雷达领域的应用。项目设计与文档由智能雷达实验室成员精心编写，旨在帮助新成员快速融入实验室研究工作。

In [ ]:
from IPython.display import HTML
# 显示CSS
HTML(css)

## Do-1 深度学习理论基础
 * 神经网络基本原理
 * 激活函数与非线性
 * 前向传播与反向传播
 * 梯度下降算法
 * 过拟合与正则化
 * 批量归一化
 * 卷积神经网络原理
 * 池化操作与降维

* 简单理解神经网络：写出一个带有未知参数的函数，这个函数可以对指定的目标任务做出预测，而该参数含有的未知参数需要通过数据去找出来。
![](eq1.png)  
* 其中y是准备要预测的东西（输入），x1为输入信息，未知参数w为<span style="color:red">权重</span>，用来控制输入信息对最终输出的影响有多大，未知参数b为<span style="color:red">偏置</span>，该带有未知参数的函数称为<span style="color:red">模型</span>。<span style="color:red">特征</span>用来指代数据中的有用信息，如图像的边缘、纹理、颜色分布、形状等，这些信息有利于模型进行预测。


* 损失函数的输入就为未知参数b、w，作用是来代表在固定好某一对b和w时，该模型的输出好还是不好，例如平均绝对误差（Mean Absolute Error，MAE）:
![](eq2.png)  
* 还有均方误差（Mean Squared Error，MSE）：  
![](eq3.png)    
* MAE、MSE用来计算预测值与你希望的输出（<span style="color:red">标签</span>）之间的差距。

In [13]:
import torch
import torch.nn as nn

# 假设模型输出和真实标签
y_predict = torch.tensor([2.5, 0.0, 2.1], requires_grad=True)
y_label = torch.tensor([3.0, -0.5, 2.0])

# MSE（均方误差）
mse_loss = nn.MSELoss()
mse = mse_loss(y_predict, y_label)

# MAE（平均绝对误差）
mae_loss = nn.L1Loss()
mae = mae_loss(y_predict, y_label)
print("MAE:", mae.item())

MAE: 0.36666664481163025


* 根据不同的b,w参数，损失函数的大小会发生变化，若能找出一对数值b、w，使得最终的损失值最小，这就是训练的目的，参数通过对输入数据的迭代学习进行更新，逐渐趋向于最优解。  
* 如何找出这个最优解，也就是学习策略，常用<span style="color:red">梯度下降法</span>，如下图：  
<p align="center">
  <img src="fig1.png" width="35%">
</p>
* 为简化问题，先解释损失值L和参数w的关系，首先随机选取一个初始点，计算该点损失值L对参数w的偏导数，就能知道w往哪个方向走（增大或减小）使得损失值L下降，并且斜率大的地方步伐就跨大一点，斜率小的地方步伐就跨小一点。  
* <span style="color:red">学习率</span>η也会影响步伐大小，是自己设定的，如果学习率设大一点，每次参数更新就会量大，学习可能会比较快，但是容易使损失在学习的过程中震荡，如果学习率设小一点，每次参数更新只会改变一点点参数的数值，但容易陷入<span style="color:red">局部最小值</span>。这种需要自己设定，而不是机器自己找出来的，称之为<span style="color:red">超参数</span>。

<p align="center">
  <img src="fig2.png" width="35%">
</p>  

* 当参数调整到在该点的梯度（偏导）为零时，有两种情况：<span style="color:red">局部最小值</span>、<span style="color:red">全局最小值</span>，参数更新在这两点都会停止，而我们希望找到的时全局最小值，但学习经常会在局部最小值点停止。
* 此外，损失函数是自己定义的，这个图中的曲线并不是一个真实的损失，这个损失的曲线可以是任何形状，而且并不仅仅只与一个或两个参数有关。例如这个模型有57个参数：  
![](eq4.png)
* 但是这个模型依旧是个简单的线性模型，输入x与输出y之间可能有复杂的关系，我们可以将输入x与y的映射关系（红色曲线）看作是一组s形状的线（蓝色）与一个常数相加之和。  
<p align="center">
  <img src="fig3.png" width="35%">
</p>    

* 并且我们常用<span style="color:red">Sigmoid函数</span>来逼近这些蓝色曲线（hard Sigmoid）：  
![](eq5.png)
<p align="center">
  <img src="fig4.png" width="35%">
</p>   

* 其中c，b，w是常数，我们可以通过改变这些常数，制造出不同的Sigmoid函数，把不同的Sigmoid函数叠加来逼近各种不同的分段函数，就可以变成任何连续的曲线。
  <p align="center">
  <img src="fig5.png" width="35%">
</p>   

* 所以在各种书上对<span style="color:red">激活函数</span>的描述为：激活函数（非线性函数）可以给神经网络增加“非线性能力”，让模型可以学习复杂的模式和关系。常见的激活函数还有<span style="color:red">ReLU</span>。在模型的各个环节增加激活函数，可以使得模型更加灵活：
  <p align="center">
  <img src="fig6.png" width="35%">
</p>   

* 在实际输入大量数据来寻找最优参数点时，通常会把这大量数据分成一个一个的<span style="color:red">批量（batch）</span>,本来是要把所有的数据拿出来算出一个损失，现在是只拿一个批量里边的数据来算出一个损失，算出梯度，来更新参数，然后继续下一个批量里边的数据算出损失、算出梯度、更新参数，当把所有数据都看过一次，称之为一个<span style="color:red">回合（epoch）</span>，每更新一次参数叫做一次更新。

* <span style="color:red">神经网络</span>由很多个神经元组成，<span style="color:red">神经元</span>通过多个输入与对应的权重相乘，加上一个偏置，然后通过一个激活函数，输出一个结果。神经元很多的模型，那些既不是模型直接接受输入的地方也不是直接输出预测结果的地方，称之为<span style="color:red">隐藏层</span>，人们把神经网络越叠越多越叠越深，例如残差网络（Residual Network,ResNet）可以有152层。
  <p align="center">
  <img src="fig7.png" width="35%">
</p>   

* 深度学习网络可以看作是一个很深层的神经网络，<span style="color:red">反向传播</span>是训练深度学习网络的核心算法之一，梯度从输出层开始，一层层“反向”传播回输入层，更新每一层的权重（参数）。而<span style="color:red">前向传播</span>：输入->经过神经网络->得到输出（预测）。

In [ ]:
import torch
import torch.nn as nn

# 1. 输入和目标（注意：需要是 float 类型）
x = torch.tensor([[0.5, -0.2]], requires_grad=True)   # 输入特征
y_true = torch.tensor([[1.0]])                        # 目标输出

# 2. 简单的线性模型：输入(2) -> 输出(1)
model = nn.Linear(2, 1)  # 自动初始化权重和偏置

# 3. 损失函数：均方误差
criterion = nn.MSELoss()

# 4. 前向传播
y_pred = model(x)

# 5. 计算损失
loss = criterion(y_pred, y_true)

# 6. 反向传播
loss.backward()

* 反向传播更新参数时，梯度值可能变得非常小或非常大，就是所谓的<span style="color:red">梯度消失</span>或<span style="color:red">梯度爆炸</span>，梯度消失会导致深层特征并没有被学习到但参数不再更新，如前边提到的局部极小值，梯度爆炸会导致参数直接变成NaN，训练无法继续，造成梯度消失和梯度爆炸的原因有很多，比如学习率过高或过低，所以学习率这种超参数需要在实验过程中慢慢调试。
* <span style="color:red">鞍点</span>的梯度也为零，但区别于局部极小值和局部极大值，如图中红色的点在y轴方向比较高，但在x轴方向是比较低的。我们把梯度为零的点统称为<span style="color:red">临界点</span>。
  <p align="center">
  <img src="fig8.png" width="35%">
</p>   

* 应对梯度消失和梯度爆炸的策略有<span style="color:red">批量归一化（Batch Normalization，BN）</span>，如今几乎是神经网络的标配，网络每一层的输出如果数值分布变化太大，会让网络训练困难，BN在每一层后把输出重新“归一化”，让数值更加平稳、集中，模型更容易收敛。

In [ ]:
import torch.nn as nn

# 用于全连接层输出
nn.BatchNorm1d(num_features)

# 用于卷积层输出（特征图）
nn.BatchNorm2d(num_channels)

* 除了批归一化以外，还有<span style="color:red">特征归一化（Feature Normalization,FN）</span>等。

* 深层网络很容易出现<span style="color:red">过拟合</span>，最明显的表现是模型在训练集上表现非常好，但在验证集和测试集上表现很差，也就是说模型记住了训练数据，而不是学会了泛化规律。为了解决该问题，很多时候采用<span style="color:red">正则化</span>的策略，通俗地讲，是给模型加上一些约束，防止它“死记硬背”，例如L1正则化和L2正则化，还有<span style="color:red">Dropout</span>，其策略是训练时随机“屏蔽”一部分神经元，防止模型对某些神经元太依赖，而测试时自动关闭Dropout。

In [ ]:
import torch.nn as nn
nn.Dropout(p=0.5)  # 每次随机丢掉50%的神经元

* 另外一个解决过拟合的策略是<span style="color:red">数据增强</span>，例如通过对训练数据做旋转、缩放、加噪声等方式，人为扩种数据多样性，让模型学会适应更多的情况，所以可以讲训练数据集越小，网络越深，越容易产生过拟合。
* 另外一个解决过拟合的策略是<span style="color:red">早停</span>，当验证集的损失不再下降时，就提前停止训练。
* 除了过拟合以外，还有一个严重问题，就是<span style="color:red">数据泄露</span>，指的是训练过程中使用了测试集的信息，或者用了模型不该提前知道的特征，你以为模型预测很准，其实它是提前“偷看了答案”才那么准，真正面对新数据时完全不行，例如测试集的数据混入训练集，用测试集做特征选择、调参、归一化等。
* 相反，过于<span style="color:red">简单的模型</span>，会产生<span style="color:red">模型偏差</span>，即模型在学习过程中的系统误差，也就是它对真实关系的理解能力差，导致预测结果总是离正确值有偏差，就比如你用一个线性模型学习一个非线性函数，无论喂多少数据，调多少次参数，模型的训练结果都不够好，因为模型本身就太简单。表现有训练误差很大，并且测试误差
* 如果给模型的<span style="color:red">限制过多</span>也会导致模型偏差。
* 除了模型本身的复杂度以外，<span style="color:red">优化做得不好</span>也容易产生模型偏差，就比如前边讲到的梯度下降，既可能找到全局最小值，也可能找到局部最小值，优化梯度下降不给力，就会容易卡在局部最小值处。常用的优化方法是自适应学习率，如<span style="color:red">Adam</span>。

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 构造训练数据
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]])

# 2. 定义模型：简单线性回归 y = wx + b
model = nn.Linear(1, 1)

# 3. 损失函数：均方误差 MSE
criterion = nn.MSELoss()

# 4. 优化器：Adam（学习率可调）
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 5. 训练过程
for epoch in range(200):
    # 前向传播
    y_pred = model(x)
    
    # 计算损失
    loss = criterion(y_pred, y)

    # 清空旧梯度
    optimizer.zero_grad()

    # 反向传播
    loss.backward()

    # 更新参数（Adam 优化）
    optimizer.step()

    # 打印训练过程
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 9.1448
Epoch 20, Loss: 5.3230
Epoch 40, Loss: 2.7615
Epoch 60, Loss: 1.2894
Epoch 80, Loss: 0.5611
Epoch 100, Loss: 0.2530
Epoch 120, Loss: 0.1420
Epoch 140, Loss: 0.1071
Epoch 160, Loss: 0.0959
Epoch 180, Loss: 0.0905


* 除此以外，还有一个问题需要补充：<span style="color:red">不匹配</span>。例如训练数据与测试数据分布不同，实际使用的数据来源不同，模型训练的优化目标与实际任务目标不一致，训练阶段和测试阶段数据预处理方式不一致（如归一化），训练使用了dropout，但测试没有给关掉，导致测试时预测根本不准，这可能不是因为过拟合的问题，可能就是以上这些不匹配问题。

* 此外卷积神经网络还有一些补充概念。
* 对于机器，图像可以描述为三维<span style="color:red">张量</span>（张量可以想象成维度大于2的矩阵），一维代表图像的宽，一维代表图像的高，还有一维代表图像的<span style="color:red">通道（channel）</span>的数目。
* 卷积神经网络会设定一个区域，即<span style="color:red">感受野</span>，每个神经元都只关心自己的感受野里面发生的事情，感受野是由我们自己决定。
  <p align="center">
  <img src="fig9.png" width="35%">
</p>   

* 我们把右上角的感受野往右移一个步幅，就制造出一个新的感受野，移动的量称之为<span style="color:red">步幅（stride）</span>，步幅是一个超参数，需要人为调整，一般不会设置太大。当感受野超出了图像的范围进行补零值，称之为<span style="color:red">填充（padding）</span>。
* 不同感受野的神经元可以进行<span style="color:red">参数共享</span>，所谓参数共享就是两个神经元的参数（权重）完全是一样的。
  <p align="center">
  <img src="fig10.png" width="35%">
</p>   

* 感受野、参数共享本质都是简化，<span style="color:red">全连接层</span>可以自己决定看整张图片还是一个小范围，加上感受野的概念后，只能看一个小范围，而加入参数共享后某一些神经元无论如何参数都要一模一样，这又增加了对神经元的限制，而感受野加上参数共享就是<span style="color:red">卷积层</span>，用到卷积层的网络就叫卷积神经网络。卷积层会改变图像的通道数而不会改变高和宽。

In [ ]:
import torch.nn as nn

# 定义一个卷积层：输入通道 3（RGB图像），输出通道 16，卷积核大小 3x3
conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

* 卷积神经网络中，<span style="color:red">池化（pooling）</span>主要用于简化特征、降低过拟合的风险，常见的池化右最大池化（Max Pooling）、平均池化（Avg Pooling）和全局平局池化（Global Avg Pooling）。池化会改变图像的高和宽而不会改变通道数。
  <p align="center">
  <img src="fig11.png" width="35%">
</p>   


In [ ]:
import torch.nn as nn

# 最大池化层：窗口大小 2x2，步长 2
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

# 平均池化层
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
通常池化层会接在卷积层之后，例如：
nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2)
)

* 池化和<span style="color:red">下采样</span>都可以简化特征，而下采样指代更广义的尺寸缩小方法，不拘泥于池化。